In [4]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: read).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in c

In [1]:
# %%
import torch
import numpy as np
import random
from transformer_lens import HookedTransformer, ActivationCache
import transformer_lens.utils as utils
import plotly.express as px

torch.set_grad_enabled(False)

SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

# %%
model = HookedTransformer.from_pretrained('gemma-7b', dtype=torch.float16)
print(model)

# %%
out = model.generate("Hi, my name is")
print(out)

# %%

n_layers = model.cfg.n_layers
d_model = model.cfg.d_model
n_heads = model.cfg.n_heads
d_head = model.cfg.d_head
d_mlp = model.cfg.d_mlp
d_vocab = model.cfg.d_vocab


# %%
prelude = "A highly knowledgeable and intelligent AI answers multiple-choice questions about Biology. "
question = "To prevent desiccation and injury, the embryos of terrestrial vertebrates are encased within a fluid secreted by the:"
answers = """
(A) amnion
(B) chorion
(C) allantois
(D) yolk sac
"""
post_text = "Answer: ("

answer_token = "A"

text = prelude + question + answers + post_text
print(model.to_str_tokens(text))

utils.test_prompt(text, answer_token, model, prepend_space_to_answer=False)
# %%
# DLA

logits, cache = model.run_with_cache(text)
cache: ActivationCache


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

...

  0%|          | 0/10 [00:00<?, ?it/s]

Hi, my name is Stacy.  I am a mental health professional offering
['<bos>', 'A', ' highly', ' knowledgeable', ' and', ' intelligent', ' AI', ' answers', ' multiple', '-', 'choice', ' questions', ' about', ' Biology', '.', ' To', ' prevent', ' desic', 'cation', ' and', ' injury', ',', ' the', ' embryos', ' of', ' terrestrial', ' vertebrates', ' are', ' encased', ' within', ' a', ' fluid', ' secreted', ' by', ' the', ':', '\n', '(', 'A', ')', ' am', 'nion', '\n', '(', 'B', ')', ' chor', 'ion', '\n', '(', 'C', ')', ' all', 'anto', 'is', '\n', '(', 'D', ')', ' yolk', ' sac', '\n', 'Answer', ':', ' (']
Tokenized prompt: ['<bos>', 'A', ' highly', ' knowledgeable', ' and', ' intelligent', ' AI', ' answers', ' multiple', '-', 'choice', ' questions', ' about', ' Biology', '.', ' To', ' prevent', ' desic', 'cation', ' and', ' injury', ',', ' the', ' embryos', ' of', ' terrestrial', ' vertebrates', ' are', ' encased', ' within', ' a', ' fluid', ' secreted', ' by', ' the', ':', '\n', '(', 'A', ')'

Performance on answer token:
Rank: 0        Logit: 25.86 Prob: 89.89% Token: |A|

Top 0th token. Logit: 25.86 Prob: 89.89% Token: |A|
Top 1th token. Logit: 22.75 Prob:  4.01% Token: |B|
Top 2th token. Logit: 22.27 Prob:  2.47% Token: |C|
Top 3th token. Logit: 21.83 Prob:  1.60% Token: |D|
Top 4th token. Logit: 21.69 Prob:  1.39% Token: |a|
Top 5th token. Logit: 18.92 Prob:  0.09% Token: |b|
Top 6th token. Logit: 18.92 Prob:  0.09% Token: |1|
Top 7th token. Logit: 18.78 Prob:  0.08% Token: |c|
Top 8th token. Logit: 18.55 Prob:  0.06% Token: |d|
Top 9th token. Logit: 18.53 Prob:  0.06% Token: | A|


Ranks of the answer tokens: [('A', 0)]

In [2]:
from datasets import load_dataset

dataset = load_dataset("cais/mmlu", "high_school_biology")

Generating test split:   0%|          | 0/310 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/32 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/5 [00:00<?, ? examples/s]

In [3]:
prompts = []
correct_answers = []

letters = ['A', 'B', 'C', 'D']

prelude = "A highly knowledgeable and intelligent AI answers multiple-choice questions about Biology. "
post_text = "Answer: ("

for i in range(100):
  question = dataset["test"][i]["question"] + ':\n'

  answers = f"""
(A) {dataset["test"][i]["choices"][0]}
(B) {dataset["test"][i]["choices"][1]}
(C) {dataset["test"][i]["choices"][2]}
(D) {dataset["test"][i]["choices"][3]}
"""

  correct_answers.append(dataset["test"][i]["answer"])

  prompts.append(prelude + question + answers + post_text)

In [4]:
print(prompts[0])

A highly knowledgeable and intelligent AI answers multiple-choice questions about Biology. In a population of giraffes, an environmental change occurs that favors individuals that are tallest. As a result, more of the taller individuals are able to obtain nutrients and survive to pass along their genetic information. This is an example of:

(A) directional selection.
(B) stabilizing selection.
(C) sexual selection.
(D) disruptive selection.
Answer: (


In [9]:
batch_size = 1

stacked_components = []

for b in range(0, 100, batch_size):
  _, cache = model.run_with_cache(model.to_tokens(prompts[b:b+batch_size]))

  decomposed, component_labels = cache.decompose_resid(layer=-1, return_labels=True, pos_slice=-1)
  stacked_mlps = decomposed[2::2, :, :]
  component_labels = component_labels[2::2]

  stacked_heads, head_labels = cache.stack_head_results(layer=-1, pos_slice=-1, return_labels=True)

  stacked_components.append(torch.cat([stacked_mlps, stacked_heads]))

component_labels.extend(head_labels)
stacked_components = torch.cat(stacked_components, dim=1) # [n_components, 100, d_model]

Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results 

In [6]:
stacked_components.shape

torch.Size([476, 100, 3072])

In [10]:
#decomposed, component_labels = cache.decompose_resid(layer=-1, return_labels=True, pos_slice=-1)
#stacked_mlps = decomposed[2::2, :, :]
#component_labels = component_labels[2::2]
#
#stacked_heads, head_labels = cache.stack_head_results(layer=-1, pos_slice=-1, return_labels=True)
#
#stacked_components = torch.cat([stacked_mlps, stacked_heads])
#component_labels.extend(head_labels)

In [31]:
letter_tokens = model.to_tokens(['A', 'B', 'C', 'D'], prepend_bos=False)

dla_letters = torch.cat([cache.logit_attrs(stacked_components[:, i], letter_tokens).to('cpu')[None] for i in range(100)], 0)

print('dla shape', dla_letters.shape)

dla shape torch.Size([100, 4, 476])


In [46]:
dla_letters.shape, torch.tensor(correct_answers)[:, None, None].expand(-1, -1, 476).shape

(torch.Size([100, 4, 476]), torch.Size([100, 1, 476]))

In [48]:
correct_letter_attr = torch.gather(dla_letters, 1, torch.tensor(correct_answers)[:, None, None].expand(-1, -1, 476))
all_letters_attr = dla_letters.mean(1)[:, None]

In [49]:
correct_letter_attr.shape, all_letters_attr.shape

(torch.Size([100, 1, 476]), torch.Size([100, 1, 476]))

In [56]:
logit_attr = (correct_letter_attr - all_letters_attr).mean(0)[0].cpu().type(torch.float32)

In [59]:
top_k_values, top_k_indices = torch.topk(logit_attr, 40)
top_labels = [component_labels[i] for i in top_k_indices]

In [60]:
fig = px.bar(y=top_k_values, x=top_labels)
fig.show()

In [61]:
top_k_indices

tensor([ 32, 458,  24,  22,  25, 366,  23, 369,   0, 432, 465,  21, 453, 365,
        410,  28, 430, 391, 400,   1, 474,  14,  60,  83, 347,   8, 392,  29,
         63,  91,  51,  85, 461,  52, 240,  56, 424, 302,  87,  37])

In [72]:
print(prompts[-1], correct_answers[-1])

A highly knowledgeable and intelligent AI answers multiple-choice questions about Biology. Which of the following theories is based on the notion that mitochondria and chloroplasts evolved from prokaryotic cells?:

(A) Fluid mosaic model
(B) Endosymbiotic model
(C) Taxonomic model
(D) Respiration feedback model
Answer: ( 1


In [66]:
from torchtyping import TensorType as TT
import einops

def stack_head_pattern_from_cache(
        cache,
    ) -> TT["layer_and_head_index", "batch", "dest_pos", "src_pos"]:
        """Stacks the head patterns from the cache into a single tensor."""
        stacked_head_pattern = torch.stack(
            [cache["pattern", l] for l in range(model.cfg.n_layers)], dim=0
        )
        stacked_head_pattern = einops.rearrange(
            stacked_head_pattern,
            "layer batch head_index dest_pos src_pos -> (layer head_index) batch dest_pos src_pos",
        )
        return stacked_head_pattern

In [67]:
patterns = stack_head_pattern_from_cache(cache).cpu()

In [68]:
patterns.shape

torch.Size([448, 1, 69, 69])

In [76]:
plot_toks = [f"{tok} ({i})" for i, tok in enumerate(model.to_str_tokens(prompts[-1]))]
fig = px.imshow(patterns[458-28, 0], x=plot_toks, y=plot_toks, aspect='auto', title='Head Pattern')
fig.update_layout(width=1400, height=1400)

In [71]:
letter_idxs = [38, 44, 50, 57]

plot_toks = [f"{tok} ({i})" for i, tok in enumerate(model.to_str_tokens(prompts[-1]))]
fig = px.imshow(patterns[458-28, 0, -1, None], x=plot_toks, aspect='auto')
fig.update_layout(width=1400, height=400)

In [ ]:
top_k_values, top_k_indices = torch.topk(dla, 10)

In [ ]:
_, large_mag_indices = torch.topk(torch.abs(dla), 20)
large_mag_values = dla[large_mag_indices]

In [ ]:
top_labels = [component_labels[i] for i in top_k_indices]

In [ ]:
fig = px.bar(y=top_k_values, x=top_labels)
fig.show()

In [ ]:
top_labels = [component_labels[i] for i in large_mag_indices]
fig = px.bar(y=large_mag_values, x=top_labels)
fig.show()

In [ ]:
correct_dir = # unembed(anwer) - mean(unembed(all))